## Goal

This notebook aims to transfer task1 to a classification problem using Word2Vec and BERT. Given the key words in specific task questions, the program is able to predict if a paper is highly related to the questions or not. 

## Aproaches

### **Generate training data**

#### **Create key words set**
 
The training data is generated by Word2Vec. We manually chose key words in task1 (i.e. *transmission, incubation, environmental stability, virus shedding *and* shedding virus*). Then we ran Word2Vec on the key word *transmission* and found top 10 words which are higly similar with it. Two sets of key words are combined as the key words set and a weight is assigned to each word to calculate the frequency score in the abstract. For the manually pulled key words, the weight is 1. For the ones generated from algorithm, the weight is the Word2Vec similarity.

The key words and the weights are as follows ('key word', weight):

[('transmission', 1),  
 ('incubation', 1),  
 ('environmental stability', 1),  
 ('virus shedding', 1),  
 ('shedding virus', 1),  
 ('spread', 0.7522934079170227),  
 ('spillover', 0.6482195854187012),  
 ('transmissibility', 0.6166895031929016),  
 ('contact', 0.6136766076087952),  
 ('transmit', 0.6054409742355347),  
 ('dispersal', 0.585004985332489),  
 ('importation', 0.551548182964325),  
 ('zoonotic', 0.5362335443496704),  
 ('epidemic', 0.5349624156951904),  
 ('movement', 0.5341535210609436)]
 
#### **Calculate the sum of key words score**

We took abstract in *metadata.csv* as context to search for key words existence. We computed the key words score by the dot of frequency in the context and the weight of each word in the key words set. 

#### **Label extreme positive and negative examples**
Then we extracted extreme positive and extreme negative examples from the dataset and gave them binary labels. 

The defination of extreme positive and negative examples are as follows:  

    Extreme positive examples: Abstracts which have key words score greater or equal than 3.   
    Extreme negative examples: Abstracts which have key words score equal to 0. 

We labeled extreme positive examples as 1 and negative examples as 0.

### **Run pre-trained BERT model and fine-tuning**

We used BERT classification model to train data and fine-tuning the parameters. 
The ratio of training set and validation set is 9:1. 
The model was run in 16 batches and 2 epoches. 
We saved the model as output and load it to run on the test set. 

### **Predict on test set**

The test set is created using body_text as context. Because body_text has longer context than abstract so the key words frequency in a body_text should be higher than that in an abstract. Since 3 is the 97th percentile of the key words score in abstracts. We found 38 as the threshold for body_text which is also the 97th percentile of the key words score in body_text. 

After generating the test set, we loaded the fine-tuned BERT model to predict if each body_text's label. 
The performance metrics is also provided as part of the result. 

Once we have the model ready, then we can use it to classify the papers which are highly related to the topic given the key words. 

### **Acknowledgement**

Some of our work is based on other people's work:

Body_text dataframe: https://www.kaggle.com/danielwolffram/cord-19-create-dataframe  
BERT classification fine-tuning: https://mccormickml.com/2019/07/22/BERT-fine-tuning/


# Word2Vec on key word *transmission*

In [ ]:
from IPython.utils import io
with io.capture_output() as captured:
    !pip install scispacy
    !pip install '/kaggle/input/pythonmodule/en_core_sci_md-0.2.4'

In [ ]:
import numpy as np 
import pandas as pd
import scispacy
import spacy
from spacy.lang.en import English
from gensim.models import Word2Vec
from tqdm import tqdm
import en_core_sci_md
import logging 

df = pd.read_csv('/kaggle/input/cord19-body-text/cord19_df.csv')

df.abstract = df.abstract.dropna()
df.abstract = df.abstract.astype(str)
abstract_doc = ' '.join(df.abstract)

nlp = English()
nlp.add_pipe(nlp.create_pipe('sentencizer'))
nlp.max_length = 40000000
doc = nlp(abstract_doc)
sentences = [sent.string.strip() for sent in doc.sents]

nlp_tkn = en_core_sci_md.load()
customize_stop_words = ['Abstract', 'article']
for w in customize_stop_words:
    nlp_tkn.vocab[w].is_stop = True

abstract_tokens = []
for sentence in tqdm(sentences):
    sent = nlp_tkn(sentence)
    token_list = [token.lemma_.lower() for token in sent if not token.is_punct | token.is_space | token.is_stop] 
    abstract_tokens.append(token_list)

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
model = Word2Vec(abstract_tokens, 
                 min_count=5,   # Ignore words that appear less than this
                 size=100,      # Dimensionality of word embeddings
                 workers=4,     # Number of processors (parallelisation)
                 window=10,     # Context window for words during training) 
                 iter=10)     

model.save('word2vec.model')
model = Word2Vec.load("word2vec.model")
model.wv.save_word2vec_format('abstract_word2vec_embeddings')
model.most_similar(positive=['transmission'], topn = 10)

# **Generat Training and Test Data**

In [ ]:
import numpy as np 
import pandas as pd 
import re

In [ ]:
def count_freq(input_text, key_word):
    '''
    This function is to calculate the frequency of each key_word in an input text.
    '''
    count = sum(1 for _ in re.finditer(r'(?i)\b%s\b' % re.escape(key_word), input_text))
    return count

def frequency_generator(key_words_weight, df, context_col):
    '''
    This function consumes the count_freq function and applies on the dataframe column (context_col) which has input text.
    It creates new columns with frequency for each key word in the dataframe. 
    '''
    key_words = [i[0] for i in key_words_weight]
    df[context_col] = df[context_col].astype(str)
    for word in key_words:
        df['count_' + str(word)] = df[context_col].apply(lambda x: count_freq(x, word))
    return df

def generate_training_data(df, context_col_name, sum_col_name, label_col_name, lower_b, neg_pos_ratio):
    '''
    This function has the following steps:
        1.Drop null and duplicated data in the context column.
        2.Generate positive examples which have at least n(lower_b) frequency of key words in the context.
        3.Generate negative examples which have none of the key words in the context. Its size is neg_pos_ratio * the size of the postive examples.
        4.Combine the positive and negative examples as the traning set and rename the label column.
    '''
    df = df.dropna(subset = [context_col_name])
    df = df.drop_duplicates(subset = [context_col_name])
    df_positive = df[df[sum_col_name] >= lower_b]
    negative_set = df[df[sum_col_name] == 0]
    df_negative = negative_set.sample(n = df_positive.shape[0] * neg_pos_ratio, replace=True, random_state=1)
    df_training = df_positive.append(df_negative)[[context_col_name, label_col_name]]
    df_training = df_training.rename(columns = {label_col_name:'label'})
    return df_training

def generate_test_data(df, context_col_name, sum_col_name, label_col_name, lower_b, upper_b, positive_size, neg_pos_ratio):
    '''
    This funtion is to generate test set which is similar as generating the training data.
    The distinctive point is the upper bound of positive set is defined in the arguements. 
    '''
    df = df.dropna(subset = [context_col_name])
    df = df.drop_duplicates(subset = [context_col_name])
    test_positive = df[(df[sum_col_name] > lower_b) & (df[sum_col_name] < upper_b)].sample(n = positive_size, replace = True, random_state = 1)
    test_negative = df[df[sum_col_name] == 0]
    test_negative = test_negative.sample(n = test_positive.shape[0] * neg_pos_ratio, replace=True, random_state=1)
    df_test = test_positive.append(test_negative)[[context_col_name, sum_col_name, label_col_name]]
    df_test = df_test.rename(columns = {label_col_name:'label'})
    return df_test

In [ ]:
root_path = '/kaggle/input/CORD-19-research-challenge'
metadata_path = f'{root_path}/metadata.csv'
metadata_df = pd.read_csv(metadata_path, dtype={
    'pubmed_id': str,
    'Microsoft Academic Paper ID': str, 
    'doi': str
})

In [ ]:
'''Key words extracted from word2vec. For key words: transmission, incubation, virus shedding and shedding virus 
existing in the first task and subtasks, the weights are 1 for each word. For the remaining ones extracted 
from word2vec, the weights are based on word2vec similarity scores.
'''
w2v_model = Word2Vec.load("/kaggle/working/word2vec.model")
w2v_model.wv.save_word2vec_format('abstract_word2vec_embeddings')
key_words_weight = w2v_model.most_similar(positive=['transmission'], topn = 10)
key_words_weight = [('transmission', 1), 
 ('incubation', 1),
 ('virus shedding', 1),
 ('shedding virus', 1),
 ('environmental stability', 1)] + key_words_weight

print(key_words_weight)
keywords_weight_dict = dict(key_words_weight)

### Generate training set on abstract

In [ ]:
neg_pos_ratio = 4 #4 means the ratio of negative examples to positive examples is 4:1
threshold = 3 #3 is the 97th percentile of the dot product of key words frequency and weights on the abstract context
meta_df = frequency_generator(key_words_weight, metadata_df, 'abstract')
col_list = ['count_' + i for i in list(keywords_weight_dict)]
meta_df['keywords_sum'] = meta_df[col_list].values.dot(pd.Series(keywords_weight_dict))
meta_df['keywords_exist'] = meta_df['keywords_sum'].apply(lambda x: 1 if x>= threshold else 0)
training_set = generate_training_data(meta_df, 'abstract', 'keywords_sum', 'keywords_exist', threshold, neg_pos_ratio)
training_set.to_csv('training_set_threshold_' + str(threshold) + '.csv')

### Generate test set on body text

In [ ]:
threshold = 38 #38 is the 97th percentile of the dot product of key words frequency and weights on the body text context
full_data = pd.read_csv('/kaggle/input/cord19-body-text/cord19_df.csv') #This dataset is the output of this notebook: https://www.kaggle.com/danielwolffram/cord-19-create-dataframe
full_text_freq = frequency_generator(key_words_weight, full_data, 'body_text')
full_text_freq['keywords_sum'] = full_text_freq[col_list].values.dot(pd.Series(keywords_weight_dict))
full_text_freq['keywords_exist'] = full_text_freq['keywords_sum'].apply(lambda x: 1 if x >= threshold else (0 if x == 0 else None))

In [ ]:
#Take the first 3000 characters of body_text as test data
full_text_freq['body_text'] = full_text_freq['body_text'].apply(lambda x: re.sub(r'[^\w\s]','',x)[:3000])
full_text_freq = full_text_freq.rename(columns = {'keywords_exist': 'label'})
full_text_freq.to_csv('body_text_with_keywords_freq_38.csv')

# Bert Classification Model
#### The model is based on this tutorial: https://mccormickml.com/2019/07/22/BERT-fine-tuning/

In [ ]:
! pip install transformers

In [ ]:
import tensorflow as tf
import torch

device_name = tf.test.gpu_device_name()
device = torch.device("cuda")
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
    print('GPU:', torch.cuda.get_device_name(0))
else:
    raise SystemError('GPU device not found')

In [ ]:
import pandas as pd
import csv
import random
import time
import datetime
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
class train_bert:
    def __init__(self, training_df, context_col, label_col):
        self.df = training_df
        self.df[context_col] = self.df[context_col].astype('str')
        self.context = self.df[context_col].values
        self.label = self.df[label_col].values
    
    def tokenizer(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        self.input_ids = []
        self.attention_masks = []
        for item in self.context:
            self.encoded_dict = self.tokenizer.encode_plus(
                    item,                      
                    add_special_tokens = True, 
                    max_length = 300,           
                    pad_to_max_length = True,
                    return_attention_mask = True,  
                    return_tensors = 'pt')
            self.input_ids.append(self.encoded_dict['input_ids'])
            self.attention_masks.append(self.encoded_dict['attention_mask'])
        self.input_ids = torch.cat(self.input_ids, dim=0)
        self.attention_masks = torch.cat(self.attention_masks, dim=0)
        self.label = torch.tensor(self.label)    
        return self
    
    def data_preparation(self, train_val_ratio): 
        self.dataset = TensorDataset(self.input_ids, self.attention_masks, self.label)
        self.train_size = int(train_val_ratio * len(self.dataset))
        self.val_size = len(self.dataset) - self.train_size
        self.train_dataset, self.val_dataset = random_split(self.dataset, [self.train_size, self.val_size])
        print('{:>5,} training samples'.format(self.train_size))
        print('{:>5,} validation samples'.format(self.val_size))
        return self
    
    def generate_data_loader(self, batch_size):
        self.batch_size = batch_size
        self.train_dataloader = DataLoader(
                    self.train_dataset,
                    sampler = RandomSampler(self.train_dataset),
                    batch_size = self.batch_size)
        self.validation_dataloader = DataLoader(
                    self.val_dataset,
                    sampler = SequentialSampler(self.val_dataset),
                    batch_size = self.batch_size)
        return self
    
    def model(self):
        self.model = BertForSequenceClassification.from_pretrained(
                    'bert-base-uncased', 
                    num_labels = 2,  
                    output_attentions = False, 
                    output_hidden_states = False)
        self.model.cuda()
        return self
    
    def optimizer(self):
        self.optimizer = AdamW(self.model.parameters(),
                  lr = 2e-5, 
                  eps = 1e-8
                )
        return self
    
    def scheduler(self, epochs):
        self.epochs = epochs
        self.total_steps = len(self.train_dataloader) * self.epochs
        self.scheduler = get_linear_schedule_with_warmup(
                    self.optimizer, 
                    num_warmup_steps = 0,
                    num_training_steps = self.total_steps)
        return self
    
    def fine_tuning(self, seed_val):
        self.seed_val = seed_val
        random.seed(self.seed_val)
        np.random.seed(self.seed_val)
        torch.manual_seed(self.seed_val)
        torch.cuda.manual_seed_all(self.seed_val)
        self.training_stats = []
        self.total_t0 = time.time()
        for epoch_i in range(0, self.epochs):
            print("")
            print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, self.epochs))
            print('Training...')
            self.t0 = time.time()
            self.total_train_loss = 0
            self.model.train()
            
            for step, batch in enumerate(self.train_dataloader):
                if step % 40 == 0 and not step == 0:
                    self.elapsed = format_time(time.time() - self.t0)
                    print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(self.train_dataloader), self.elapsed))
                self.b_input_ids = batch[0].to(device)
                self.b_input_mask = batch[1].to(device)
                self.b_labels = batch[2].to(device)
                self.model.zero_grad()        

                self.loss, self.logits = self.model(self.b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=self.b_input_mask, 
                             labels=self.b_labels)
                self.total_train_loss += self.loss.item()
                self.loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
                self.optimizer.step()
                self.scheduler.step()
            self.avg_train_loss = self.total_train_loss / len(self.train_dataloader)        
            self.training_time = format_time(time.time() - self.t0)
            print("")
            print("  Average training loss: {0:.2f}".format(self.avg_train_loss))
            print("  Training epcoh took: {:}".format(self.training_time))

            print("")
            print("Running Validation...")

            self.t0 = time.time()
            self.model.eval()

            self.total_eval_accuracy = 0
            self.total_eval_loss = 0
            self.nb_eval_steps = 0

            for batch in self.validation_dataloader:
                self.b_input_ids = batch[0].to(device)
                self.b_input_mask = batch[1].to(device)
                self.b_labels = batch[2].to(device)
                with torch.no_grad():
                    (self.loss, self.logits) = self.model(self.b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=self.b_input_mask,
                                   labels=self.b_labels)
                self.total_eval_loss += self.loss.item()
                self.logits = self.logits.detach().cpu().numpy()
                self.label_ids = self.b_labels.to('cpu').numpy()
                self.total_eval_accuracy += flat_accuracy(self.logits, self.label_ids)
            self.avg_val_accuracy = self.total_eval_accuracy / len(self.validation_dataloader)
            print("  Accuracy: {0:.3f}".format(self.avg_val_accuracy))
            self.avg_val_loss = self.total_eval_loss / len(self.validation_dataloader)
            self.validation_time = format_time(time.time() - self.t0)
            print("  Validation Loss: {0:.3f}".format(self.avg_val_loss))
            print("  Validation took: {:}".format(self.validation_time))
            self.training_stats.append(
                {
            'epoch': epoch_i + 1,
            'Training Loss': self.avg_train_loss,
            'Valid. Loss': self.avg_val_loss,
            'Valid. Accur.': self.avg_val_accuracy,
            'Training Time': self.training_time,
            'Validation Time': self.validation_time
                }
            )
        
        print("")
        print("Training complete!")
        print("Total training took {:} (h:mm:ss)".format(format_time(time.time()- self.total_t0)))
        return self
    
    def save_model(self, file_name):
        self.file_name = file_name
        torch.save(self.model, self.file_name)

In [ ]:
df = pd.read_csv('/kaggle/working/training_set_threshold_3.csv', engine = 'python', sep= ',', header=0)
bert = train_bert(df, 'abstract', 'label' )
token = bert.tokenizer().data_preparation(0.9).generate_data_loader(16).model().optimizer().scheduler(2).fine_tuning(42)

In [ ]:
token.save_model('bert_model_3')

In [ ]:
model_train = torch.load('/kaggle/working/bert_model_3')

In [ ]:
test_df = pd.read_csv('/kaggle/input/fulltestbodytext38/full_test_body_text_38.csv')

In [ ]:
class prediction:
    
    def __init__(self, test_df, context_col, label_col, model_train):
        self.df = test_df
        self.context_col = context_col
        self.model_train = model_train
        self.df[context_col] = self.df[context_col].astype('str')
        self.context = self.df[context_col].values
        self.label = self.df[label_col].values
        print('Number of {}: {:,}\n'.format(context_col, df.shape[0]))
    
    def test_tokenizer(self):
        train_bert.tokenizer(self)
        return self
    
    def prediction(self, batch_size):
        self.batch_size = batch_size
        self.prediction_data = TensorDataset(self.input_ids, self.attention_masks, self.label)
        self.prediction_sampler = SequentialSampler(self.prediction_data)
        self.prediction_dataloader = DataLoader(self.prediction_data, sampler=self.prediction_sampler, batch_size=self.batch_size)
        print('Predicting labels for {:,} test {}...'.format(len(self.input_ids), self.context_col))
        self.model_train.eval()
        self.predictions , self.true_labels = [], []
        for batch in self.prediction_dataloader:
            batch = tuple(t.to(device) for t in batch)
            self.b_input_ids, self.b_input_mask, self.b_labels = batch
            with torch.no_grad():
                self.outputs = self.model_train(self.b_input_ids, token_type_ids=None, 
                      attention_mask=self.b_input_mask)
            self.logits = self.outputs[0]
            self.logits = self.logits.detach().cpu().numpy()
            self.label_ids = self.b_labels.to('cpu').numpy()
            self.predictions.append(self.logits)
            self.true_labels.append(self.label_ids)
        print('    DONE.')
        return self

In [ ]:
prediction = prediction(test_df, 'body_text', 'label', model_train).test_tokenizer().prediction(16)

In [ ]:
flat_predictions = np.concatenate(prediction.predictions, axis=0)
flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
flat_true_labels = np.concatenate(prediction.true_labels, axis=0)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(flat_true_labels, flat_predictions)

In [ ]:
tn, fp, fn, tp = confusion_matrix(flat_true_labels, flat_predictions).ravel()
precision = tp / (tp + fp)
recall = tp / (tp + fn)
print('The precision is {}'.format(precision))
print('The recall is {}'.format(recall))